# CTA Train Data Investigation

In this notebook I want to do a bit of exploration regarding using the data provided by the Chicago Transit Authority detailing the riding history of the trains since 2001. This will be used in a generative music project. In particular, make sure that you have the python package PyAudio installed (I used conda to install this package)

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date

In [2]:
cta_ride_data = pd.read_csv('cta-boarding-totals.csv')
cta_info = pd.read_csv('cta-system-info.csv')

In [3]:
cta_ride_data

,station_id,stationname,date,daytype,rides
0,40350,UIC-Halsted,01/01/2001,U,273
1,41130,Halsted-Orange,01/01/2001,U,306
2,40760,Granville,01/01/2001,U,1059
3,40070,Jackson/Dearborn,01/01/2001,U,649
4,40090,Damen-Brown,01/01/2001,U,411
5,40590,Damen/Milwaukee,01/01/2001,U,870
6,40720,East 63rd-Cottage Grove,01/01/2001,U,391
7,41260,Austin-Lake,01/01/2001,U,399
8,40230,Cumberland,01/01/2001,U,788
9,41120,35-Bronzeville-IIT,01/01/2001,U,448


In [4]:
update_date = lambda row: datetime.strptime(row['date'], '%m/%d/%Y')
cta_ride_data['date'] = cta_ride_data.apply(update_date, axis=1)
cta_ride_data.head()

,station_id,stationname,date,daytype,rides
0,40350,UIC-Halsted,2001-01-01,U,273
1,41130,Halsted-Orange,2001-01-01,U,306
2,40760,Granville,2001-01-01,U,1059
3,40070,Jackson/Dearborn,2001-01-01,U,649
4,40090,Damen-Brown,2001-01-01,U,411


In [5]:
cta_ride_data.sort_values(['station_id', 'date'], inplace=True)
cta_ride_data.head(15)

,station_id,stationname,date,daytype,rides
61,40010,Austin-Forest Park,2001-01-01,U,290
243,40010,Austin-Forest Park,2001-01-02,W,1240
315,40010,Austin-Forest Park,2001-01-03,W,1412
434,40010,Austin-Forest Park,2001-01-04,W,1388
609,40010,Austin-Forest Park,2001-01-05,W,1465
833,40010,Austin-Forest Park,2001-01-06,A,613
894,40010,Austin-Forest Park,2001-01-07,U,403
1054,40010,Austin-Forest Park,2001-01-08,W,1463
1263,40010,Austin-Forest Park,2001-01-09,W,1505
1320,40010,Austin-Forest Park,2001-01-10,W,1519


In [6]:
cta_info.head()

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,...,Pexp,Y,Pnk,O,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,...,False,False,True,False,"(41.857908, -87.669147)",8.0,14920,33.0,343.0,26.0
1,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,...,False,False,True,False,"(41.857908, -87.669147)",8.0,14920,33.0,343.0,26.0
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,...,False,False,False,True,"(41.829353, -87.680622)",26.0,14924,56.0,719.0,1.0
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,...,False,False,False,True,"(41.829353, -87.680622)",26.0,14924,56.0,719.0,1.0
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,...,False,False,False,False,"(41.831677, -87.625826)",12.0,21194,1.0,25.0,9.0


Let's cut out the relevant columns that we need from the dataframe containing the station information.

In [7]:
# cut out the relevant info
cta_info = cta_info.loc[:,['STATION_NAME', 'MAP_ID', 'Location']]

In [8]:
cta_info.head(10)

,STATION_NAME,MAP_ID,Location
0,18th,40830,"(41.857908, -87.669147)"
1,18th,40830,"(41.857908, -87.669147)"
2,35th/Archer,40120,"(41.829353, -87.680622)"
3,35th/Archer,40120,"(41.829353, -87.680622)"
4,35th-Bronzeville-IIT,41120,"(41.831677, -87.625826)"
5,35th-Bronzeville-IIT,41120,"(41.831677, -87.625826)"
6,43rd,41270,"(41.816462, -87.619021)"
7,43rd,41270,"(41.816462, -87.619021)"
8,47th,41080,"(41.809209, -87.618826)"
9,47th,41080,"(41.809209, -87.618826)"


Because the station information dataframe contained entries for unique stop-ids, the above dataframe contains duplicate information. Let's remove the duplicates based on unique station_ids. We'll also do some renaming of our columns and change the index so that we can access stations by their ID.

In [9]:
# now let's reduce it so that we only have unique locations and ids
# as can be seen above, the data now contains duplicates (different stop-ids)
# are not as relevant to us.
cta_info.drop_duplicates('MAP_ID', inplace=True)
cta_info.rename(columns={'MAP_ID': 'station_id', 
                 'STATION_NAME': 'stationname', 
                 'Location': 'location'},
                 inplace=True)
cta_info.set_index('station_id', inplace=True)
cta_info.head(10)

,stationname,location
station_id,,
40830,18th,"(41.857908, -87.669147)"
40120,35th/Archer,"(41.829353, -87.680622)"
41120,35th-Bronzeville-IIT,"(41.831677, -87.625826)"
41270,43rd,"(41.816462, -87.619021)"
41080,47th,"(41.809209, -87.618826)"
41230,47th,"(41.810318, -87.63094)"
40130,51st,"(41.80209, -87.618487)"
40580,54th/Cermak,"(41.85177331, -87.75669201)"
40910,63rd,"(41.780536, -87.630952)"


Now, we need to do a bit of parsing on the location field. In particular, the field looks like it should be a tuple of floats, but is currently expressed as a string. We can use the ast package to do literal_evaluation of the field, offloading some of the heavy lifting here to get back a tuple of floats representing the GPS location of the stations. We'll then save the latitude and longitude into new columns and remove the now superfluous column of tuples.

In [10]:
from ast import literal_eval

def convert_loc(row):
    location = literal_eval(row.location)
    return location

cta_info['location'] = cta_info.apply(convert_loc, axis=1)

get_lat = lambda r: r.location[0]
get_long = lambda r: r.location[1]

cta_info['latitude'] = cta_info.apply(get_lat, axis=1)
cta_info['longitude'] = cta_info.apply(get_long, axis=1)
cta_info.drop(columns = 'location', inplace=True)

We can see the changes reflected in the dataframe of station information.

In [11]:
cta_info.head()

,stationname,latitude,longitude
station_id,,,
40830,18th,41.857908,-87.669147
40120,35th/Archer,41.829353,-87.680622
41120,35th-Bronzeville-IIT,41.831677,-87.625826
41270,43rd,41.816462,-87.619021
41080,47th,41.809209,-87.618826


Now, we want to use the latitude and longitude of each station to map to particular control parameters for the overall sound. We'll first make some NumPy arrays from the latitude and longitude, so that we may perform calculations using the data.

Fantastic, now we simply have to identify what parameters should be mapped using the location and ride data that we have collected. In particular, we have the following main fields that we can use: latitude, longitude, and rideship over time per station. Perhaps most obviously, we could map the latitude of a station to a particular frequency and the rideship over time to amplitude at that frequency over time, but the question of what to do with the longitude is less obvious. Theoretically, we could perhaps adjust tuning or harmonic inclusion using this field, but some further experimentation will be required.

Moreover, we will import my own set of tools defined in the np_audio script that will help us with some of the interpolation and signal composition needed.

In [12]:
import pyaudio
import np_audio

SAMPLE_RATE = 441000
length = 300

p = pyaudio.PyAudio()

In [13]:
p.open(format=pyaudio.paFloat32, channels=1, rate=44100, output=True, output_device_index=1)

In [14]:
cta_info_sorted = cta_info.sort_values('station_id')
freq_llimit = 70
freq_ulimit = 15000
cta_info_sorted['frequency'] = np_audio.interpolate_to_frequency(cta_info_sorted.latitude.to_numpy(),
                                                                freq_llimit, freq_ulimit)
cta_info_sorted.head()

,stationname,latitude,longitude,frequency
station_id,,,,
40010,Austin,41.870851,-87.776812,6389.465471
40020,Harlem/Lake,41.886848,-87.803176,7070.342184
40030,Pulaski,41.885412,-87.725404,7009.222039
40040,Quincy/Wells,41.878723,-87.633740,6724.519636
40050,Davis,42.047710,-87.683543,13917.075313


Let's now create a new pandas dataframe that can hold the entirety of our location information as well as the associated numpy array that holds mapped amplitudes over time from the ridership information.

In [16]:
audio_df = cta_info_sorted.drop(columns = 'stationname')

def audio_map_wrap(row):
    """ Given a row of a dataframe, finds the associated ride information and 
        gets a mapped numpy array representing amplitude values. """
    station_id = row.name
    rides = cta_ride_data.loc[cta_ride_data['station_id'] == station_id, 'rides']
    amps = rides.to_numpy()
    return np_audio.interpolate_to_amplitude(amps)

def date_map_wrap(row):
    """ Given a row of a dataframe, gets the date information and maps to a new
        numpy array based on the desired length and sample rate. """
    station_id = row.name
    dates = cta_ride_data.loc[cta_ride_data['station_id'] == station_id, 'date']
    return np_audio.interpolate_datetime(dates.to_numpy(), length, SAMPLE_RATE)

audio_df['amplitude'] = audio_df.apply(audio_map_wrap, axis=1)
audio_df['sample_locs'] = audio_df.apply(date_map_wrap, axis=1)

In [17]:
audio_df.head(10)

,latitude,longitude,frequency,amplitude,sample_locs
station_id,,,,,
40010,41.870851,-87.776812,6389.465471,"[0.0338927278710102, 0.3464955577492596, 0.403...","[0, 20127, 40255, 60383, 80511, 100638, 120766..."
40020,41.886848,-87.803176,7070.342184,"[0.08887147335423198, 0.45203761755485894, 0.4...","[0, 20127, 40255, 60383, 80511, 100638, 120766..."
40030,41.885412,-87.725404,7009.222039,"[0.03968978102189781, 0.3804744525547445, 0.45...","[0, 20127, 40255, 60383, 80511, 100638, 120766..."
40040,41.878723,-87.633740,6724.519636,"[0.0380894184743864, 0.7879621142682555, 0.819...","[0, 20127, 40255, 60383, 80511, 100638, 120766..."
40050,42.047710,-87.683543,13917.075313,"[0.1520423600605144, 0.6049546142208775, 0.657...","[0, 20127, 40255, 60383, 80511, 100638, 120766..."
40060,41.938132,-87.712359,9253.131543,"[0.15434552199258084, 0.5360360360360361, 0.55...","[0, 20127, 40255, 60383, 80511, 100638, 120766..."
40070,41.878183,-87.629296,6701.535738,"[0.03526215702254822, 0.3138820972561804, 0.35...","[0, 20127, 40255, 60383, 80511, 100638, 120766..."
40080,41.953775,-87.654929,9918.941034,"[0.05003509475584077, 0.3245763561616364, 0.35...","[0, 20127, 40255, 60383, 80511, 100638, 120766..."
40090,41.966286,-87.678639,10451.443913,"[0.1024682124158564, 0.45450012465719275, 0.47...","[0, 20127, 40255, 60383, 80511, 100638, 120766..."


Alright, now that we have saved the associated amplitude arrays in our pandas dataframe for easy retrieval, we can begin to perform operations. Let's quickly check to make sure that everything is still ok.

In [19]:
def check_length(row):
    return len(row.amplitude) == len(row.sample_locs)

audio_df.apply(check_length, axis=1).describe()

count      143
unique       1
top       True
freq       143
dtype: object